# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
data_file = "../output_data/cities.csv"
dataframe = pd.read_csv(data_file)


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# filter for cities with ideal weather condition for travel planning (condition slightly adjusted from instructor to have a reasonable number of cities)
hotel_df = dataframe.loc[(dataframe["Max Temp"] > 270) & 
(dataframe["Max Temp"] < 283) & (dataframe["Wind Speed"] < 50) & (dataframe["Cloudiness"] < 70)]
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
40,40,Bohdan,11,UA,1655343455,86,48.0412,24.3522,282.55,1.00
43,43,Lampa,11,CL,1655343456,76,-33.2833,-70.9000,282.39,2.06
84,84,Riobamba,42,EC,1655343468,93,-1.6667,-78.6333,281.88,1.73
182,182,Leżajsk,49,PL,1655343495,95,50.2626,22.4193,282.71,2.34
210,210,Skibotn,47,NO,1655343504,81,69.3913,20.2679,281.69,1.83
274,274,Klimovo,0,RU,1655343519,94,52.3805,32.1923,282.52,2.98
331,331,Yasinya,27,UA,1655343534,91,48.2794,24.3604,281.80,0.55
334,334,Buchmany,65,UA,1655343534,94,51.0689,28.0421,282.52,2.55
343,343,Colquechaca,0,BO,1655343537,19,-18.6667,-66.0167,271.87,1.57
368,368,Irati,7,BR,1655343543,97,-25.4672,-50.6511,282.83,0.04


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
names = []
hotel_data = []

for index, row in hotel_df.iterrows():
    # geocoordinates
    target_coordinates = str(row["Lat"]) + ',' + str(row["Lng"])
    target_radius = 5000
    target_search = "hotel"
    target_type = "hotel"

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "key_word": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    places_data = response.json()
    hotel_data.append(places_data)
    names.append(places_data["results"][0]["name"])
    print(json.dumps(places_data, indent=4, sort_keys=True))



{
    "html_attributions": [],
    "next_page_token": "Aap_uEClyv5myaHotqd9Ph1T2AZxH9yWd37ElvZWoIiXjk69U4hC3V2rtdpJAZbePqUYRdJiwbWPl30DAZy4jTbaCgeBds5RxuoelSReh9T4HratmjfdcWRKQrjAfuphLvsRkWaaUPmpTjEm6guQVOKbkvx90mAsahl7Icnz0YMCsLzRzDAY7lE3Ao2SliT5mmIdXtys4n891Rr_TeIBehUrUDNp6FXpTXMeZmvOZ1PPV2irVeZEVQwFtxZWQEMeDkxL9AgcsC6mkDMBpjDJCaFNJmj9xZL09Tyq3cKeA-SEH50WiTKvD2VcnQ6yNqnInLLrpN0PoEdoNhsrw3i8VHA1MrNfNzlkDFBD3lg4vVpTygIYi2KczpCj0Kra5XMXbY3uC8KMfSJgtraXEbyQWOMPPKtTXLUmQlHiDinSrqqC-3uCNZ7Mtb_97w",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 48.0392785,
                    "lng": 24.3456792
                },
                "viewport": {
                    "northeast": {
                        "lat": 48.04984698745739,
                        "lng": 24.37537006464793
                    },
                    "southwest": {
                        "lat": 48.02188000353873,
                        "lng": 24.32511496199993


In [7]:
hotel_df["Hotel Name"] = names

C:\Users\BeauM\AppData\Local\Temp/ipykernel_28372/2889517300.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = names


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
# Add marker layer ontop of heat map
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(dataframe[["Lat", "Lng"]], weights=dataframe["Humidity"], 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)
marker_locations =[]

for hotel in hotel_data:  
  marker_locations.append((hotel["results"][0]["geometry"]["location"]["lat"],hotel["results"][0]["geometry"]["location"]["lng"]))

markers = gmaps.marker_layer(marker_locations,info_box_content=hotel_info)
fig.add_layer(markers)
fig

# Display figure



Figure(layout=FigureLayout(height='420px'))